# TlF B state Hamiltonian
## Intro
This notebook evaluates the Hamiltonian for the B state of thallium fluoride.

In [3]:
import numpy as np
import sympy
from sympy import sqrt
import multiprocessing
import pickle

### Representing the states

Import class that represents molecular states from 'molecular-state-classes-and-functions'

In [4]:
import sys
sys.path.append('../molecular-state-classes-and-functions/')

from classes import *
from functions import *

### Defining operators

### Rotational term

The simplest term in the Hamiltonian simply gives the rotational levels:

$$H_\text{rot}=B_\text{rot}\vec J^2.$$

In [5]:
def J2(psi):
    return State([(psi.J*(psi.J+1),psi)])

def Hrot(psi):
    return Brot * J2(psi)

### Electron magnetic hyperfine operator
Since the B state is a triplet pi state, it has electron magnetic hyperfine structure. The coupling is described by the Hamiltonian $ H_{\mathrm{mhf}} = a\, \mathbf{I} \cdot \mathbf{L} + b\, \mathbf{I} \cdot \mathbf{S} +c \,I_z\, S_z  $ which reduces to $ H_{\mathrm{mhf}}^{eff} =  [a \,L_z + (b+c)\, S_z]\,I_z = h_\Omega \, I_z $ since the raising and lowering operators for L and S (electron orbital and spin angular momentum) only couple different electronic states which are very far in energy and thus the effect of the off-diagonal elements is strongly suppressed. The matrix elements are given in eqns 5 and 6 in Norggard et al 2017.

In [21]:
#Import Wigner 3j symbol
from sympy.physics.wigner import wigner_3j, wigner_6j

def H_mhf_Tl(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    I1 = psi.I1
    I2 = psi.I2
    F1 = psi.F1
    F = psi.F
    mF = psi.mF
    omega = 1
    
    #I1, I2, F1 and F and mF are the same for both states
    I1prime = I1
    I2prime = I2
    F1prime = F1
    mFprime = mF
    Fprime = F
    
    #Container for the states and amplitudes
    data = []
    
    #Loop over possible values of Jprime
    for Jprime in np.arange(np.abs(J-1),J+2):

        #Check that the Jprime and Fprime values are physical
        if np.abs(Fprime-Jprime) <= (I1+I2):
            #Calculate matrix element
            try:
                amp = h1_Tl*((-1)**(J+Jprime+F1+I1-omega) 
                       * wigner_6j(I1, Jprime, F1, J, I1, 1) 
                       * wigner_3j(J, 1, Jprime, -omega, 0, omega)
                       * sqrt((2*J+1)*(2*Jprime+1)*I1*(I1+1)*(2*I1+1)))

            except ValueError: 
                amp = 0

            basis_state = CoupledBasisState(Fprime, mFprime, F1prime, Jprime, I1prime, I2prime)

            #If matrix element is non-zero, add to list
            if amp != 0:
                data.append((amp, basis_state))
                       
    return State(data)
            

In [22]:
def H_mhf_F(psi):
    #Find the quantum numbers of the input state
    J = psi.J
    I1 = psi.I1
    I2 = psi.I2
    F1 = psi.F1
    F = psi.F
    mF = psi.mF
    omega = 1
    
    #I1, I2, F and mF are the same for both states
    I1prime = I1
    I2prime = I2
    Fprime = F
    mFprime = mF
    
    #Initialize container for storing states and matrix elements
    data = []
    
    #Loop over the possible values of quantum numbers for which the matrix element can be non-zero
    #Need Jprime = J+1 ... |J-1|
    for Jprime in np.arange(np.abs(J-1), J+2):
        
        #Loop over possible values of F1prime
        for F1prime in np.arange(np.abs(Jprime-I1), Jprime+I1+1):
            try:
                amp = h1_F*((-1)**(2*F1prime+F+2*J+1+I1+I2-omega) 
                       * wigner_6j(I2, F1prime, F, F1, I2, 1)
                       * wigner_6j(Jprime, F1prime, I1, F1, J, 1) 
                       * wigner_3j(J, 1, Jprime,-omega,0,omega)
                       * sqrt((2*F1+1)*(2*F1prime+1)*(2*J+1)*(2*Jprime+1)*I2*(I2+1)*(2*I2+1)))

            except ValueError: 
                amp = 0

            basis_state = CoupledBasisState(Fprime, mFprime, F1prime, Jprime, I1prime, I2prime)

            #If matrix element is non-zero, add to list
            if amp != 0:
                data.append((amp, basis_state))

        
    return State(data)

In [23]:
def Hff(psi):
    return Hrot(psi) + H_mhf_Tl(psi) + H_mhf_F(psi)

### Finding the matrix elements

With all the operators defined, we can evaluate the matrix elements for a given range of quantum numbers. We shall need to generate a non-integer range list (e.g., from -3/2 to +3/2):

In [24]:
def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(sympy.Number(x))
        x += dx
    return range_list

Define constants for TlF:

In [25]:
half = sympy.Rational(1,2)
#half = 0.5

Jmin = sympy.Integer(1)
Jmax = sympy.Integer(6) # max J value in Hamiltonian
#Jmax = 6
I_Tl = half             # I1 in Ramsey's notation
I_F  = half             # I2 in Ramsey's notation

Brot = sympy.symbols('Brot')
h1_Tl, h1_F = sympy.symbols('h1_Tl h1_F')

 Write down the basis as a list of `BasisState` components:

In [26]:
QN = [CoupledBasisState(F,mF,F1,J,I_F,I_Tl)
      for J  in ni_range(Jmin, Jmax+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for mF in ni_range(-F,F+1)
     ]

The field-free and Stark/Zeeman components of the Hamiltonian then have the matrix elements (evaluate using `multiprocessing` to speed things up)

In [27]:
%%time
from tqdm import tqdm

def HMatElems(H, QN=QN):
    result = sympy.zeros(len(QN),len(QN))
    for i,a in tqdm(enumerate(QN)):
        for j,b in enumerate(QN):
            result[i,j] = (1*a)@H(b)
            
    return result

H_ops = [Hff]
Hff_m = HMatElems(Hff)

192it [05:01,  1.59s/it]


Wall time: 5min 1s


In [105]:
Hff_m

Matrix([
[2*Brot - h1_F/2 - h1_Tl/2,                         0,          h1_F/6 - h1_Tl/2,                         0,                                 0,                   -sqrt(2)*h1_F/6,                                 0,                                  0,                                  0,                                  0,                                  0,                                  0,                                 0,                   -sqrt(6)*h1_F/6,                                 0,                                  0,                                  0,                                  0,                                  0,                                  0,                                 0,                                 0,                                 0,                                 0,                                 0,                                  0,                                  0,                                  0,                              

Store the result of the calculation as text files and Python `pickle`s:

In [28]:
with open("hamiltonians_symbolic_coupled.py", 'wb') as f:
    pickle.dump(
        {
            "Hff" : Hff_m,
        },
        f
    )

In [29]:
with open("hamiltonians_symbolic_coupled.txt", 'w') as f:
    f.write(
        str(
            {
                "Hff" : Hff_m,
            }
        )
    )

## Testing magnetic hyperfine

In [20]:
Ja = 1
I1 = 1/2
I2 = 1/2
F1a = 1/2
Fa = 0
mFa = 0

a = CoupledBasisState(Fa, mFa, F1a, Ja, I1, I2)

Jb = 1
I1 = 1/2
I2 = 1/2
F1b = 1/2
Fb = 1
mFb = 0
b = CoupledBasisState(Fb, mFb, F1b, Jb, I1, I2)



print((1*b) @ H_mhf_Tl(a))

print((1*b) @ H_mhf_F(a))


-0.204124145231932*sqrt(6)*h1_Tl
-0.204124145231932*sqrt(6)*h1_F


In [ ]:
wigner_6j()